## Dia 4

Hoje criaremos um modelo de recomendações de filmes para um usuário qualquer, a partir do dataset 'MovieLens'.

In [1]:
import pandas as pd

In [3]:
df_filmes = pd.read_csv('../DS_Dia_4/u.item', sep='|', names=['movie id', 'movie title', 'release date', 'video release date',
                 'IMDb URL', 'unknown', 'Action', 'Adventure', 'Animation', 'Childrens', 'Comedy', 'Crime', 'Documentary', 
                 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western'],
                 encoding='ISO-8859-1')

df_filmes = df_filmes.set_index(keys='movie id')
df_filmes.head()

,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Childrens,Comedy,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movie id,,,,,,,,,,,,,,,,,,,,,
1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


Como só queremos avaliar um filme, iremos dropar a coluna 'release date', 'video release date' e 'IMDb URL' de noso DataFrame

In [ ]:
df_filmes = df_filmes.drop(columns=['release date', 'video release date', 'IMDb URL'])
df_filmes.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1682 entries, 1 to 1682
Data columns (total 20 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   movie title  1682 non-null   object
 1   unknown      1682 non-null   int64 
 2   Action       1682 non-null   int64 
 3   Adventure    1682 non-null   int64 
 4   Animation    1682 non-null   int64 
 5   Childrens    1682 non-null   int64 
 6   Comedy       1682 non-null   int64 
 7   Crime        1682 non-null   int64 
 8   Documentary  1682 non-null   int64 
 9   Drama        1682 non-null   int64 
 10  Fantasy      1682 non-null   int64 
 11  Film-Noir    1682 non-null   int64 
 12  Horror       1682 non-null   int64 
 13  Musical      1682 non-null   int64 
 14  Mystery      1682 non-null   int64 
 15  Romance      1682 non-null   int64 
 16  Sci-Fi       1682 non-null   int64 
 17  Thriller     1682 non-null   int64 
 18  War          1682 non-null   int64 
 19  Western      1682 non-null   int

In [ ]:
df_filmes.duplicated().sum()

np.int64(18)

In [ ]:
df_filmes = df_filmes.reset_index()
df_filmes.duplicated().sum()

np.int64(0)

Como podemos ver nossos dados não possuem valores nulos, mas valores duplicados quando a coluna 'movie title' é o índice do DataFrame, porém não iremos retirar ela de nossa tabela, pois perderíamos informações de avalições para nossos filmes mesmo com índices distintos.

In [ ]:
df_filmes.head()

,movie id,movie title,unknown,Action,Adventure,Animation,Childrens,Comedy,Crime,Documentary,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),0,0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0


Para gerarmos nosso modelo de recomendação precisamos importar os usuários e suas respectivamos notas e colocaremos tudo em um único DataFrame

In [ ]:
df_usuarios = pd.read_csv('./Dia_4/u.user', sep='|', names=['user id', 'age', 'gender', 'occupation', 'zip code'], encoding='ISO-8859-1')
df_usuarios = df_usuarios.set_index('user id')
df_usuarios

,age,gender,occupation,zip code
user id,,,,
1,24,M,technician,85711
2,53,F,other,94043
3,23,M,writer,32067
4,24,M,technician,43537
5,33,F,other,15213
...,...,...,...,...
939,26,F,student,33319
940,32,M,administrator,02215
941,20,M,student,97229


In [ ]:
df_usuarios.info()

<class 'pandas.core.frame.DataFrame'>
Index: 943 entries, 1 to 943
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   age         943 non-null    int64 
 1   gender      943 non-null    object
 2   occupation  943 non-null    object
 3   zip code    943 non-null    object
dtypes: int64(1), object(3)
memory usage: 36.8+ KB


In [ ]:
df_usuarios.duplicated().sum()

np.int64(7)

In [ ]:
df_usuarios.reset_index()
df_usuarios = df_usuarios.drop('zip code', axis=1)
df_usuarios.head()

,age,gender,occupation
user id,,,
1,24,M,technician
2,53,F,other
3,23,M,writer
4,24,M,technician
5,33,F,other


In [ ]:
df_avaliacoes = pd.read_csv('./Dia_4/u.data', sep='\t', names=['user id', 'item id', 'rating', 'timestamp'], encoding='ISO-8859-1')
df_avaliacoes

,user id,item id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
...,...,...,...,...
99995,880,476,3,880175444
99996,716,204,5,879795543
99997,276,1090,1,874795795
99998,13,225,2,882399156


In [ ]:
df_avaliacoes = df_avaliacoes.drop('timestamp', axis=1)
df_avaliacoes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype
---  ------   --------------   -----
 0   user id  100000 non-null  int64
 1   item id  100000 non-null  int64
 2   rating   100000 non-null  int64
dtypes: int64(3)
memory usage: 2.3 MB


In [ ]:
df_avaliacoes.duplicated().sum()

np.int64(0)

Novamente o mesmo problema ocorre para o DataFrame dos usuários, porém pelo mesmo motivos não retiraremos dele. Por outro lado, não temos problemas com valores nulos e duplicados tanto em usuários como em avaliacoes

In [ ]:
df_user_aval = pd.merge(df_avaliacoes, df_usuarios, on='user id')
df_user_aval.head()

,user id,item id,rating,age,gender,occupation
0,196,242,3,49,M,writer
1,186,302,3,39,F,executive
2,22,377,1,25,M,writer
3,244,51,2,28,M,technician
4,166,346,1,47,M,educator


In [ ]:
df_user_aval.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   user id     100000 non-null  int64 
 1   item id     100000 non-null  int64 
 2   rating      100000 non-null  int64 
 3   age         100000 non-null  int64 
 4   gender      100000 non-null  object
 5   occupation  100000 non-null  object
dtypes: int64(4), object(2)
memory usage: 4.6+ MB


In [ ]:
df_user_aval.duplicated().sum()

np.int64(0)

Antes de criarmos o modelo vamos separar o genero e a ocupação utilizando o OneHotEncoder

In [ ]:
df_user_aval['gender'] = df_user_aval['gender'].replace({'M': 1, 'F': 0})
df_user_aval.head()

C:\Users\luizh\AppData\Local\Temp\ipykernel_10620\1438402274.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_user_aval['gender'] = df_user_aval['gender'].replace({'M': 1, 'F': 0})


,user id,item id,rating,age,gender,occupation
0,196,242,3,49,1,writer
1,186,302,3,39,0,executive
2,22,377,1,25,1,writer
3,244,51,2,28,1,technician
4,166,346,1,47,1,educator


In [ ]:
df_user_aval['occupation'].unique()

array(['writer', 'executive', 'technician', 'educator', 'engineer',
       'librarian', 'programmer', 'administrator', 'student', 'retired',
       'other', 'doctor', 'marketing', 'artist', 'lawyer', 'salesman',
       'homemaker', 'healthcare', 'none', 'entertainment', 'scientist'],
      dtype=object)

In [ ]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(categories=[['writer', 'executive', 'technician', 'educator', 'engineer',
       'librarian', 'programmer', 'administrator', 'student', 'retired',
       'other', 'doctor', 'marketing', 'artist', 'lawyer', 'salesman',
       'homemaker', 'healthcare', 'none', 'entertainment', 'scientist']], sparse_output=False)

In [ ]:
encoded_occupation = encoder.fit_transform(df_user_aval[['occupation']])

df_encoded = pd.DataFrame(encoded_occupation, columns=encoder.get_feature_names_out(['occupation']))

df_user_aval = pd.concat([df_user_aval, df_encoded], axis=1).drop('occupation', axis=1)
df_user_aval.head()


,user id,item id,rating,age,gender,occupation_writer,occupation_executive,occupation_technician,occupation_educator,occupation_engineer,...,occupation_doctor,occupation_marketing,occupation_artist,occupation_lawyer,occupation_salesman,occupation_homemaker,occupation_healthcare,occupation_none,occupation_entertainment,occupation_scientist
0,196,242,3,49,1,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,186,302,3,39,0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,22,377,1,25,1,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,244,51,2,28,1,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,166,346,1,47,1,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Vamos criar nosso modelo de recomendação

In [ ]:
filme_copia = df_filmes.copy()
filme_copia = filme_copia.rename(columns={'movie id': 'item id'})

df_treino = df_user_aval.copy()
df_treino = pd.merge(df_treino, filme_copia, on='item id')
df_treino = df_treino.drop(['item id','movie title'], axis=1)
df_treino.head()

,user id,rating,age,gender,occupation_writer,occupation_executive,occupation_technician,occupation_educator,occupation_engineer,occupation_librarian,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,196,3,49,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,186,3,39,0,0.0,1.0,0.0,0.0,0.0,0.0,...,0,1,0,0,1,0,0,1,0,0
2,22,1,25,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,244,2,28,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0,0,0,0,0,1,0,0,1,1
4,166,1,47,1,0.0,0.0,0.0,1.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
df_quantidade_filmes = df_treino.groupby('user id').sum()[['unknown', 'Action',
       'Adventure', 'Animation', 'Childrens', 'Comedy', 'Crime', 'Documentary',
       'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery',
       'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']]


In [ ]:
df_treino.iloc[:, 25:] = df_treino.iloc[:, 25:].multiply(df_treino.iloc[:, 1], axis=0)
df_treino = df_treino.drop('rating', axis=1)
df_treino.head()

,user id,age,gender,occupation_writer,occupation_executive,occupation_technician,occupation_educator,occupation_engineer,occupation_librarian,occupation_programmer,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,196,49,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,186,39,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0,3,0,0,3,0,0,3,0,0
2,22,25,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,244,28,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,2,0,0,2,2
4,166,47,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
quantidade_filmes = df_treino.groupby('user id').sum()
quantidade_filmes = quantidade_filmes.drop(['age', 'gender', 'occupation_writer', 'occupation_executive',
                                            'occupation_technician', 'occupation_educator', 'occupation_engineer',
                                            'occupation_librarian', 'occupation_programmer',
                                            'occupation_administrator', 'occupation_student', 'occupation_retired',
                                            'occupation_other', 'occupation_doctor', 'occupation_marketing',
                                            'occupation_artist', 'occupation_lawyer', 'occupation_salesman',
                                            'occupation_homemaker', 'occupation_healthcare', 'occupation_none',
                                            'occupation_entertainment', 'occupation_scientist'], axis=1)

quantidade_filmes.head()

,unknown,Action,Adventure,Animation,Childrens,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
user id,,,,,,,,,,,,,,,,,,,
1,4,250,123,40,55,316,86,24,420,7,5,45,38,18,173,172,188,92,22
2,0,38,13,4,12,61,34,0,134,3,9,6,3,14,66,15,43,11,0
3,0,39,14,0,0,31,30,5,64,0,5,12,4,35,17,22,53,14,0
4,0,31,14,0,0,20,19,5,27,0,0,4,5,20,13,23,43,9,0
5,4,176,107,53,71,246,35,0,72,5,5,71,40,9,44,116,56,45,5


In [ ]:
quantidade_filmes = quantidade_filmes.div(df_quantidade_filmes).round(2)
quantidade_filmes = quantidade_filmes.fillna(0)

df_quantidade_filmes.columns = ['v_unknown', 'v_Action',
       'v_Adventure', 'v_Animation', 'v_Childrens', 'v_Comedy', 'v_Crime', 'v_Documentary',
       'v_Drama', 'v_Fantasy', 'v_Film-Noir', 'v_Horror', 'v_Musical', 'v_Mystery',
       'v_Romance', 'v_Sci-Fi', 'v_Thriller', 'v_War', 'v_Western']

quantidade_filmes = pd.concat([quantidade_filmes, df_quantidade_filmes], axis=1)
quantidade_filmes.head()

,unknown,Action,Adventure,Animation,Childrens,Comedy,Crime,Documentary,Drama,Fantasy,...,v_Fantasy,v_Film-Noir,v_Horror,v_Musical,v_Mystery,v_Romance,v_Sci-Fi,v_Thriller,v_War,v_Western
user id,,,,,,,,,,,,,,,,,,,,,
1,4.0,3.33,2.93,3.33,2.20,3.47,3.44,4.8,3.93,3.5,...,2,1,13,13,5,44,43,52,25,6
2,0.0,3.80,4.33,4.00,3.00,3.81,3.78,0.0,3.83,3.0,...,1,2,2,1,4,16,4,12,3,0
3,0.0,2.79,3.50,0.00,0.00,2.58,3.00,5.0,2.91,0.0,...,0,2,5,2,11,5,8,21,5,0
4,0.0,3.88,3.50,0.00,0.00,5.00,4.75,5.0,4.50,0.0,...,0,0,1,1,5,3,6,11,2,0
5,4.0,3.14,3.24,3.79,2.45,3.00,3.89,0.0,2.67,2.5,...,2,1,28,12,3,19,33,19,14,2


In [ ]:
df_treino = df_treino.drop(['unknown', 'Action',
                            'Adventure', 'Animation', 'Childrens', 'Comedy', 'Crime', 'Documentary',
                            'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery',
                            'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western'], axis=1)

df_treino = df_treino.groupby('user id').mean()

df_treino = pd.concat([df_treino, quantidade_filmes], axis=1)
df_treino.head()

,age,gender,occupation_writer,occupation_executive,occupation_technician,occupation_educator,occupation_engineer,occupation_librarian,occupation_programmer,occupation_administrator,...,v_Fantasy,v_Film-Noir,v_Horror,v_Musical,v_Mystery,v_Romance,v_Sci-Fi,v_Thriller,v_War,v_Western
user id,,,,,,,,,,,,,,,,,,,,,
1,24.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,2,1,13,13,5,44,43,52,25,6
2,53.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,2,2,1,4,16,4,12,3,0
3,23.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,2,5,2,11,5,8,21,5,0
4,24.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0,0,1,1,5,3,6,11,2,0
5,33.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2,1,28,12,3,19,33,19,14,2


In [ ]:
df_treino = df_treino.reset_index().drop(['user id','age'], axis=1)
df_treino.head()

,gender,occupation_writer,occupation_executive,occupation_technician,occupation_educator,occupation_engineer,occupation_librarian,occupation_programmer,occupation_administrator,occupation_student,...,v_Fantasy,v_Film-Noir,v_Horror,v_Musical,v_Mystery,v_Romance,v_Sci-Fi,v_Thriller,v_War,v_Western
0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2,1,13,13,5,44,43,52,25,6
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,2,2,1,4,16,4,12,3,0
2,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,2,5,2,11,5,8,21,5,0
3,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,1,1,5,3,6,11,2,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2,1,28,12,3,19,33,19,14,2


In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

def avaliacao(dados):
  inercia = []
  silhueta = []

  for k in range(2,10):
    kmeans = KMeans(n_clusters=k, random_state=3165)
    kmeans.fit(dados)
    inercia.append(kmeans.inertia_)
    silhueta.append(f'k={k} - '+ str(silhouette_score(dados, kmeans.predict(dados))))
  return silhueta, inercia

Vamos fazer as medições para ver o quão bem está a previsão.

In [ ]:
silhueta, inertia_ = avaliacao(df_treino)
silhueta

['k=2 - 0.5995364525529399',
 'k=3 - 0.5055912868981787',
 'k=4 - 0.44950496523251676',
 'k=5 - 0.40933979795008646',
 'k=6 - 0.4162848292401071',
 'k=7 - 0.3237239112412848',
 'k=8 - 0.2713160257205313',
 'k=9 - 0.2567107185890734']

In [ ]:
inertia_

[2108083.0864920225,
 1441721.6964325567,
 1181565.6895805874,
 1053304.514099489,
 932387.9585076971,
 860356.4429443595,
 812743.5698199493,
 741542.0860718092]

In [ ]:
modelo_kmeans = KMeans(n_clusters=2, random_state=45)
modelo_kmeans.fit(df_treino)

df_treino['cluster'] = modelo_kmeans.labels_

cluster_media = df_treino.groupby('cluster').mean()

cluster_media.T

cluster,0,1
gender,0.774648,0.691781
occupation_writer,0.046948,0.047945
occupation_executive,0.028169,0.035616
occupation_technician,0.051643,0.021918
occupation_educator,0.089202,0.104110
occupation_engineer,0.084507,0.067123
occupation_librarian,0.061033,0.052055
occupation_programmer,0.079812,0.067123
occupation_administrator,0.084507,0.083562
occupation_student,0.215962,0.205479


In [ ]:
silhueta, inercia_ = avaliacao(df_treino)
silhueta

['k=2 - 0.5995502969357782',
 'k=3 - 0.5055880510713497',
 'k=4 - 0.44950601631577636',
 'k=5 - 0.40935435286287825',
 'k=6 - 0.41628917044075897',
 'k=7 - 0.3237203578505332',
 'k=8 - 0.27132049166993016',
 'k=9 - 0.2567097568324527']

O que é possível perceber que nosso modelo independe da ocupação do nosso usuário e da coluna 'unknown' de filmes, portanto excluiremos e executaremos novamente o modelo.

In [ ]:
df_user_aval = pd.merge(df_avaliacoes, df_usuarios, on='user id')
df_user_aval.head()

,user id,item id,rating,age,gender,occupation
0,196,242,3,49,M,writer
1,186,302,3,39,F,executive
2,22,377,1,25,M,writer
3,244,51,2,28,M,technician
4,166,346,1,47,M,educator


In [ ]:
filme_copia = df_filmes.copy()
filme_copia = filme_copia.rename(columns={'movie id': 'item id'})

df_treino = pd.merge(df_user_aval, filme_copia, on='item id')
df_treino = df_treino.drop(['item id','movie title', 'occupation'], axis=1)
df_treino['gender'] = df_treino['gender'].replace({'M': 1, 'F': 0})
df_treino.head()

C:\Users\luizh\AppData\Local\Temp\ipykernel_10620\2604793518.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_treino['gender'] = df_treino['gender'].replace({'M': 1, 'F': 0})


,user id,rating,age,gender,unknown,Action,Adventure,Animation,Childrens,Comedy,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,196,3,49,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,186,3,39,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,1,0,0
2,22,1,25,1,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
3,244,2,28,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,1
4,166,1,47,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
df_treino = df_treino.drop(['rating', 'unknown', 'Action',
                            'Adventure', 'Animation', 'Childrens', 'Comedy', 'Crime', 'Documentary',
                            'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery',
                            'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western'], axis=1)

df_treino = df_treino.groupby('user id').mean()

df_treino = pd.concat([df_treino, quantidade_filmes], axis=1)
df_treino = df_treino.reset_index().drop(['user id', 'unknown', 'age',' v_unknown'], axis=1)
df_treino.head()

,gender,Action,Adventure,Animation,Childrens,Comedy,Crime,Documentary,Drama,Fantasy,...,v_Fantasy,v_Film-Noir,v_Horror,v_Musical,v_Mystery,v_Romance,v_Sci-Fi,v_Thriller,v_War,v_Western
0,1.0,3.33,2.93,3.33,2.20,3.47,3.44,4.8,3.93,3.5,...,2,1,13,13,5,44,43,52,25,6
1,0.0,3.80,4.33,4.00,3.00,3.81,3.78,0.0,3.83,3.0,...,1,2,2,1,4,16,4,12,3,0
2,1.0,2.79,3.50,0.00,0.00,2.58,3.00,5.0,2.91,0.0,...,0,2,5,2,11,5,8,21,5,0
3,1.0,3.88,3.50,0.00,0.00,5.00,4.75,5.0,4.50,0.0,...,0,0,1,1,5,3,6,11,2,0
4,0.0,3.14,3.24,3.79,2.45,3.00,3.89,0.0,2.67,2.5,...,2,1,28,12,3,19,33,19,14,2


In [ ]:
silhueta, inertia_ = avaliacao(df_treino)
silhueta

['k=2 - 0.5996981965911726',
 'k=3 - 0.5058646702761216',
 'k=4 - 0.4498459957848427',
 'k=5 - 0.4097192405884768',
 'k=6 - 0.41667088983031364',
 'k=7 - 0.3241979822370632',
 'k=8 - 0.27186251629532926',
 'k=9 - 0.2572735188145057']

In [ ]:
inertia_

[2107123.758305066,
 1440764.6524578384,
 1180608.6361347837,
 1052349.530940106,
 931436.8737085758,
 859406.2615633123,
 811795.0788171914,
 740597.425707216]

In [ ]:
mod_kmeans = KMeans(n_clusters=2, random_state=3165)
mod_kmeans.fit(df_treino)
df_treino['cluster'] = mod_kmeans.labels_

cluster_media = df_treino.groupby('cluster').mean()

cluster_media.T.sort_values(0, ascending=False)[[0]]

cluster,0
v_Drama,25.639077
v_Comedy,17.214383
v_Action,16.302578
v_Thriller,14.922659
v_Romance,12.672999
v_Adventure,8.443691
v_Sci-Fi,8.310719
v_War,6.522388
v_Crime,5.413840
v_Childrens,4.119403


In [ ]:
cluster_media.T.sort_values(1, ascending=False)[[1]]

cluster,1
v_Drama,101.936893
v_Comedy,83.228155
v_Action,65.893204
v_Thriller,52.786408
v_Romance,49.131068
v_Adventure,36.553398
v_Sci-Fi,32.063107
v_War,22.286408
v_Childrens,20.126214
v_Crime,19.733010


Apesar de ainda baixo a silhueta, vemos que o modelo apresentou uma leve melhora. 

Além do mais, até o presente momento não foram estudados modelos mais para uma análise mais profunda, pois estamos no meio do processo de aprendizado

A partir disso pegaremos um usuário e fazeremos uma previsão com ele no arquivo em que faremos nosso modelo de recomendação.

In [ ]:
import joblib

joblib.dump(mod_kmeans, 'modelo_kmeans.pkl')

['modelo_kmeans.pkl']